In [3]:
import optuna
import joblib
import pandas as pd
import numpy as np
import warnings
import re

from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from pathlib import Path

In [2]:
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [4]:
DATA_PATH = '../data/FinalTrainingData.parquet'
MODEL_PATH = '../models/lgbm_regressor.pkl'
N_TRIALS = 50
SEED = 999
CATEGORICALS = [
    'DISTRICT_CODE', 'DISTRICT_NAME', 'LOCATION_ID',
    'SCHOOL_NAME', 'STUDENT_ID', 'ETHNIC_CODE', 
    'STUDENT_GENDER', 'ECONOMIC_CODE', 'SPECIAL_ED_CODE',
    'ENG_PROF_CODE', 'HISPANIC_IND'
]
OBJECT_COLS = [
    'Att_Rate_Prev1_2019', 'Att_Rate_Prev2_2019', 'Delta_Attendance_2019', 'Grade_X_Gender_2019', 'Econ_X_Grade_Band_2019', 'Att_Rate_Prev2_2020', 'Grade_X_Gender_2020', 'Econ_X_Grade_Band_2020', 'Grade_X_Gender_2021', 'Econ_X_Grade_Band_2021', 'Grade_X_Gender_2022', 'Econ_X_Grade_Band_2022',
    'Grade_X_Gender_2023', 'Econ_X_Grade_Band_2023'
]
FOLDS = [
    (2021, 2022),
    (2022, 2023), 
    (2023, 2024)
]

YEAR_PATTERN = re.compile(r'_(\d{4})$')

In [5]:
df = pd.read_parquet(DATA_PATH)
for col in CATEGORICALS:
    if col in df.columns:
        df[col] = df[col].astype('category')

for col in OBJECT_COLS:
    if col in df.columns:
        df[col] = df[col].astype('category')

STATIC_FEATURES = [c for c in CATEGORICALS if c in df.columns]

In [6]:
def cols_up_to(year: int) -> list[str]:
    keep = []
    for col in df.columns:
        res = YEAR_PATTERN.search(col)
        if res:
            if int(res.group(1)) <= year:
                keep.append(col)
        elif col in STATIC_FEATURES or col in OBJECT_COLS:
            keep.append(col)

    return keep

In [93]:
# Optuna Hyperparameter Tuning
def objective(trial: optuna.Trial) -> float:
    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'randome_state': SEED,
        'verbosity': -1,
        'n_jobs': -1,
        'boosting_type': 'gbdt',
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'n_estimators': trial.suggest_int('n_estimators', 50, 2000),
        'num_leaves': trial.suggest_int('num_leaves', 10, 300),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'min_child_weight': trial.suggest_float('min_child_weight', 1e-3, 10.0, log=True),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 10.0, log=True),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_gain_to_split': trial.suggest_float('min_gain_to_split', 0, 15),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-8, 10.0, log=True),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-8, 10.0, log=True),
    }

    rmses = []
    for max_feature_year, target_year in FOLDS:
        feature_cols = cols_up_to(max_feature_year)
        y_col = f'Attendance_Rate_{target_year}'

        x_full, y_full = df.loc[:, feature_cols], df.loc[:, y_col]
        x_train, x_test, y_train, y_test = train_test_split(x_full, y_full, test_size=0.2, random_state=SEED)

        model = LGBMRegressor(**params)
        model.fit(
            x_train, y_train,
            categorical_feature=[c for c in STATIC_FEATURES if c in feature_cols],
            eval_set=[(x_test, y_test)],
            eval_metric='rmse',
        )
        preds = model.predict(x_test, num_iteration=model.best_iteration_)
        rmses.append(mean_squared_error(y_test, preds)) #type: ignore

    return float(np.mean(rmses))

In [94]:
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=SEED))
study.optimize(objective, n_trials=N_TRIALS, show_progress_bar=True)
print(f'\nBest RMSE: {study.best_value}')
print(f'\nBest Hyper Parameters: {study.best_params}')

[I 2025-06-06 19:10:45,998] A new study created in memory with name: no-name-c6d4ef0b-a91f-4a4e-9a11-aa73e5e9617c
Best trial: 0. Best value: 0.00624653:   2%|▏         | 1/50 [00:13<11:05, 13.59s/it]

[I 2025-06-06 19:10:59,587] Trial 0 finished with value: 0.006246525052779368 and parameters: {'learning_rate': 0.15373071059435936, 'n_estimators': 1079, 'num_leaves': 44, 'max_depth': 11, 'min_child_samples': 13, 'min_child_weight': 0.021325671898157497, 'reg_alpha': 7.021477798505748e-05, 'reg_lambda': 0.0009760505530639855, 'feature_fraction': 0.7768759134865365, 'bagging_fraction': 0.818435766516609, 'bagging_freq': 6, 'min_gain_to_split': 1.9783552934191388, 'lambda_l1': 1.2160013712944446e-05, 'lambda_l2': 6.516831580591641e-07}. Best is trial 0 with value: 0.006246525052779368.


Best trial: 0. Best value: 0.00624653:   4%|▍         | 2/50 [00:19<07:04,  8.85s/it]

[I 2025-06-06 19:11:05,119] Trial 1 finished with value: 0.006681390413531126 and parameters: {'learning_rate': 0.11086734675956622, 'n_estimators': 115, 'num_leaves': 274, 'max_depth': 8, 'min_child_samples': 78, 'min_child_weight': 0.07852962128617527, 'reg_alpha': 3.806012947026401e-06, 'reg_lambda': 0.05776069349473615, 'feature_fraction': 0.4582539660608414, 'bagging_fraction': 0.6474146729872345, 'bagging_freq': 2, 'min_gain_to_split': 5.8541667088453835, 'lambda_l1': 0.6918037096270477, 'lambda_l2': 5.3852523055348285e-08}. Best is trial 0 with value: 0.006246525052779368.


Best trial: 0. Best value: 0.00624653:   6%|▌         | 3/50 [00:28<07:07,  9.09s/it]

...

[I 2025-06-06 19:51:58,852] Trial 47 finished with value: 0.0063257194363402726 and parameters: {'learning_rate': 0.08430604335131794, 'n_estimators': 1243, 'num_leaves': 25, 'max_depth': 11, 'min_child_samples': 31, 'min_child_weight': 1.5997084937956134, 'reg_alpha': 0.00024520472722203276, 'reg_lambda': 3.342994251494737, 'feature_fraction': 0.608676825012145, 'bagging_fraction': 0.9734504618933622, 'bagging_freq': 6, 'min_gain_to_split': 3.0018967823192946, 'lambda_l1': 1.4780108777982124e-07, 'lambda_l2': 1.9091074216763577e-06}. Best is trial 12 with value: 0.005718913501970226.


Best trial: 12. Best value: 0.00571891:  98%|█████████▊| 49/50 [43:02<01:06, 66.42s/it]

[I 2025-06-06 19:53:48,010] Trial 48 finished with value: 0.005794540933851574 and parameters: {'learning_rate': 0.026513083674087338, 'n_estimators': 1559, 'num_leaves': 119, 'max_depth': 15, 'min_child_samples': 9, 'min_child_weight': 2.7209492717316848, 'reg_alpha': 0.12400611110458828, 'reg_lambda': 2.078188319736683e-06, 'feature_fraction': 0.657924985926996, 'bagging_fraction': 0.8907223747654229, 'bagging_freq': 1, 'min_gain_to_split': 0.11784866849473072, 'lambda_l1': 1.8251128154887132e-05, 'lambda_l2': 9.603971308774499e-05}. Best is trial 12 with value: 0.005718913501970226.


Best trial: 12. Best value: 0.00571891: 100%|██████████| 50/50 [44:38<00:00, 53.57s/it]

[I 2025-06-06 19:55:24,557] Trial 49 finished with value: 0.006204809117435803 and parameters: {'learning_rate': 0.028181155569310886, 'n_estimators': 1833, 'num_leaves': 119, 'max_depth': 8, 'min_child_samples': 9, 'min_child_weight': 0.0019761825470037036, 'reg_alpha': 0.16021441030804406, 'reg_lambda': 4.838718588017718e-07, 'feature_fraction': 0.6679868663721523, 'bagging_fraction': 0.8453281080857056, 'bagging_freq': 2, 'min_gain_to_split': 1.8830353349004003, 'lambda_l1': 1.727408864932214e-05, 'lambda_l2': 0.0018140103487993132}. Best is trial 12 with value: 0.005718913501970226.

Best RMSE: 0.005718913501970226

Best Hyper Parameters: {'learning_rate': 0.03187793347826908, 'n_estimators': 1416, 'num_leaves': 15, 'max_depth': 11, 'min_child_samples': 9, 'min_child_weight': 5.0989767957043455, 'reg_alpha': 0.0065283875392360136, 'reg_lambda': 1.1449128977564993, 'feature_fraction': 0.6019754679595203, 'bagging_fraction': 0.974121394316118, 'bagging_freq': 1, 'min_gain_to_split': 

In [ ]:
study_best_params = {'learning_rate': 0.03187793347826908, 'n_estimators': 1416, 'num_leaves': 15, 'max_depth': 11, 'min_child_samples': 9, 'min_child_weight': 5.0989767957043455, 'reg_alpha': 0.0065283875392360136, 'reg_lambda': 1.1449128977564993, 'feature_fraction': 0.6019754679595203, 'bagging_fraction': 0.974121394316118, 'bagging_freq': 1, 'min_gain_to_split': 0.003244787478175533, 'lambda_l1': 1.492591506646833e-08, 'lambda_l2': 1.9818051331341124e-05}

best_params = {
    **study.best_params,
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'random_state': SEED
}

final_feature_cols = cols_up_to(2023)
final_target = 'Attendance_Rate_2024'


final_model = LGBMRegressor(**best_params)
final_model.fit(
    df[final_feature_cols],
    df[final_target],
    categorical_feature= [c for c in STATIC_FEATURES if c in final_feature_cols]
)


NameError: name 'study' is not defined